# PLN com Embedding em VectorDB 

In [5]:
# Bibliotecas para PLN 

 

# Para chunks e embeddings 

from langchain_text_splitters import RecursiveCharacterTextSplitter # pip install langchain-text-splitters

from sentence_transformers import SentenceTransformer # pip install sentence_transformers 

 

# Para leitura de PDF 

import pdfplumber # pip install pdfplumber 

 

# Para tratamento de texto 

import re 

import spacy # python -m spacy download pt_core_news_sm 

import nltk # pip install nltk 

from nltk.corpus import stopwords 

# Baixando dados do NLTK necessários (se ainda não tiver) 

nltk.download('stopwords') # rodar apenas uma vez 

AttributeError: partially initialized module 'nltk' has no attribute 'data' (most likely due to a circular import)

# Bibliotecas para Banco de Dados Vetorial (Vector Database) 

In [3]:
import chromadb 

from langchain_chroma import Chroma 

# Aplicando PLN - Preparação do texto 

In [4]:
# Carregando PDFs 

 

def ler_pdf(caminho_pdf):
    leitor_pdf = pdfplumber.open(caminho_pdf) 

# page = leitor_pdf.pages[0]
    texto = "" 

    for pagina in range(len(leitor_pdf.pages)):
        texto += leitor_pdf.pages[pagina].extract_text()
    texto = texto.replace("\n", " ") 

    return texto 
 

# Carregar os documentos do PDF 

arquivo_pdf = "/Users/ALEKS/Downloads/ML/introducaoml.pdf" 

texto_pdf = ler_pdf(arquivo_pdf) 

 

# Tamanho do texto 

print("Tamanho do texto em caracteres:",len(texto_pdf)) 


# Arquivo PDF original 

print(texto_pdf) 


# PLN 

# Carregar o modelo de linguagem do spaCy 

nlp = spacy.load("pt_core_news_sm") # Definir stopwords 

api_stop_words = set(stopwords.words('portuguese')) 

minhas_stop_words = {'a','e','i','o', 'u'} 

stop_words = api_stop_words | minhas_stop_words 



Tamanho do texto em caracteres: 913
Título: Introdução ao Machine Learning Machine learning é um campo da inteligência artificial que desenvolve algoritmos capazes de aprender padrões a partir de dados. Os principais tipos incluem aprendizado supervisionado, aprendizado não supervisionado e aprendizado por reforço. Aprendizado supervisionado envolve treinar um modelo com dados rotulados, como prever preços de casas com base em características como tamanho, localização e número de quartos. Aprendizado não supervisionado detecta padrões ocultos em dados não rotulados, como segmentação de clientes em marketing. Aprendizado por reforço ensina agentes a tomar decisões em ambientes dinâmicos para maximizar recompensas. Redes neurais profundas são usadas em visão computacional e processamento de linguagem natural. Modelos como CNNs e Transformers têm revolucionado tarefas como tradução automática, reconhecimento de imagens e geração de texto.


# Função para fazer o tratamento de linguagem natural usando spaCy 

In [5]:
def tratamento_pln(texto):
    texto = texto.lower() # 1. Normalização: Colocar o texto em minúsculas
    
    texto = re.sub(r'[^a-zA-Záéíóú\s]', '', texto) # na expressão regular estão as exceções  2. Remoção de números, pontuações e caracteres especiais 
    
    doc = nlp(texto) # 3. Tokenização com spaCy
    
    tokens = [token.text for token in doc]

    clean_tokens = [token.lemma_ for token in doc if token.text not in stop_words and not token.is_punct]  # 4. Remoção de stopwords, remoção de pontuação e Lematização (clean_tokens = tokens lematizados e sem stopwords)
    
    clean_text = ' '.join(clean_tokens) # 5. Juntar tokens lematizados de volta em uma string
    
    return clean_text 

    #return texto 

    # Visualizando as Stop Words 

print("Tamanho do conjunto stop_words:",len(stop_words),"\nStop_words ordenadas: \n",sorted(list(stop_words))) 

Tamanho do conjunto stop_words: 209 
Stop_words ordenadas: 
 ['a', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'essa', 'essas', 'esse', 'esses', 'esta', 'estamos', 'estar', 'estas', 'estava', 'estavam', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéramos', 'estivéssemos', 'estou', 'está', 'estávamos', 'estão', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'forem', 'formos', 'fosse', 'fossem', 'fui', 'fôramos', 'fôssemos', 'haja', 'hajam', 'hajamos', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houveram', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houvermos', 'houverá', 'houverão', 'houveríamos', 'houvesse', 'houvessem', '

# Visualizando as Stop Words 

In [6]:


# Chamada de PLN 

texto_pdf_tratado = tratamento_pln(texto_pdf) 

 

# Tamanho do texto 

print("Tamanho do texto em caracteres:",len(texto_pdf)) 

 

# Arquivo PDF tratado 

print(texto_pdf_tratado) 

 

# Tamanho do texto tratado 

print("Tamanho do texto em caracteres:",len(texto_pdf_tratado)) 

Tamanho do texto em caracteres: 913
título introduo machine Learning machine learning campo inteligncia artificial desenvolver algorit capaz aprender padre partir dado principal tipo incluir aprendizar supervisionar aprendizar supervisionado aprendizar reforo aprendizado supervisionar envolver treinar modelo dado rotular prever preo casa base característica tamanho localizao número quarto aprendizar supervisionado detectar padre oculto dado rotulado segmentao cliente marketing aprendizar reforo ensina agente tomar decises ambiente dinmico maximizar recompenso rede neural profundo so usar viso computacional processamento linguagem natural modelo cnns transformer tm revolucionar tarefa traduo automático reconhecimento imagem gerao texto
Tamanho do texto em caracteres: 708


# Preparação do Texto para BD Vetorial 

In [1]:
# Dividindo os documentos em Chunks 

text_splitter = RecursiveCharacterTextSplitter(chunk_size=150, chunk_overlap=30) 

chunks = text_splitter.split_text(texto_pdf_tratado) 

print(chunks, len(chunks)) 

 

# Carregar o modelo de Embeddings bem como gerar os Embeddings 

model = SentenceTransformer('all-MiniLM-L6-v2') 

embeddings = model.encode(chunks) 

 

# Gerando IDs automaticamente 

uids = [f"doc_{i}" for i in range(len(chunks))] 

NameError: name 'RecursiveCharacterTextSplitter' is not defined

## Aplicação do ChromaDB 

In [8]:
# Criar o banco de dados 

client = chromadb.Client() 

collection = client.create_collection(name="machlrn") 

 

#client.delete_collection("machlrn") 

 

# Adicionar os documentos ao banco de dados 

collection.add(documents=chunks, embeddings=embeddings, ids=uids) 

 

# Realizar a busca usando collection.query 

query_embedding = model.encode(["O que significa agir racionalmente?"]) 

query_embedding = model.encode(["Em 1965, que programas já existiam?"]) 
 
 

results = collection.query(query_embeddings=query_embedding, n_results=1) 

print(results) 

 

# Imprimir os resultados 

# Fazendo a varredura sobre os campos 'ids', 'distances' e 'documents' 

for i in range(len(results['ids'][0])):
    doc_id = results['ids'][0][i]
    
    distance = results['distances'][0][i]
    
    document = results['documents'][0][i] 

    print(f"ID: {doc_id}")
    
    print(f"Distância: {distance}")
    
    print(f"Documento: {document}")
    
    print("-" * 40) 

{'ids': [['doc_1']], 'embeddings': None, 'documents': [['partir dado principal tipo incluir aprendizar supervisionar aprendizar supervisionado aprendizar reforo aprendizado supervisionar envolver treinar']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[None]], 'distances': [[1.1930792331695557]]}
ID: doc_1
Distância: 1.1930792331695557
Documento: partir dado principal tipo incluir aprendizar supervisionar aprendizar supervisionado aprendizar reforo aprendizado supervisionar envolver treinar
----------------------------------------
